In [1]:
import numpy as np
import pandas as pd
import math
import time
import matplotlib.pyplot as plt
from scipy import spatial
from scipy.stats.stats import pearsonr

In [3]:
train=pd.read_csv('movielenstrain.csv',usecols=[1,2,3])
test=pd.read_csv('movielenstest.csv',usecols=[1,2,3])
holdout=pd.read_csv('movilensholdout.csv',usecols=[1,2,3])

In [4]:
train.head()

,movieId,rating,userId
0,301,3.0,551.0
1,1889,5.0,46.0
2,2212,5.0,586.0
3,1143,5.0,232.0
4,820,2.0,99.0


In [5]:
user_total=np.zeros(672)
cnt=np.zeros(672,dtype='int64')

In [6]:
for i,j in train.iterrows():
    k=int(j['rating'])
    cnt[k]+=1

In [7]:
movies_mean=np.zeros(shape=[9126,3])

In [8]:
user_total=np.zeros(672)
cnt=np.zeros(672,dtype='int64')
for i,j in train.iterrows():
    k=int(j['userId'])
    cnt[k]+=1
    user_total[k]+=j['rating']
    p=int(j['movieId'])
    movies_mean[p][0]+=1
    movies_mean[p][1]+=j['rating']

In [9]:
user_mean=np.array([ i/j for i,j in zip(user_total,cnt) ])
for x in movies_mean:
    x[2]=round(x[1]/x[0],2)
    
for x in movies_mean:
    if math.isnan(x[2]):
        x[2]=0
for i in range(len(user_mean)):
    if math.isnan(user_mean[i]):
        user_mean[i]=0

C:\Users\ADMIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """Entry point for launching an IPython kernel.
C:\Users\ADMIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:

t1 = time.time()

predicted_rating=[];
K=50
rmse=[[0 for i in range(10)] for j in range(10)]
Lambda=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
delta=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for index,i in holdout.iterrows():
    
    usr=int(i['userId'])
    mov=int(i['movieId'])
    actual=i['rating']
    
    list_user = train[train['userId']==usr]
    list_item = train[train['movieId']==mov]
    
    ############### SUR #########################
    similarity1=[]
    corres_rate1=[]
    similar_users=[]
    for ind2,u in list_item.iterrows():
        
        c1=[]
        c2=[]
        c_user=int(u['userId'])
        list_curuser=train[train['userId']==c_user]
        xt=pd.merge(list_user,list_curuser,how='inner',on=['movieId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity1.append(sim)
        corres_rate1.append(u['rating'])
        similar_users.append(u['userId'])
    
    corres_rate1 = [x for _,x in sorted(zip(similarity1,corres_rate1),reverse=True)]
    similar_users=[x for _,x in sorted(zip(similarity1,similar_users),reverse=True)]
    
    similarity1.sort(reverse=True)
    
    corres_rate1=corres_rate1[0:K]
    similarity1=similarity1[0:K]
    similar_users=similar_users[0:K]
    
    
    ############### SIR #########################
    similarity2=[]
    corres_rate2=[]
    similar_items=[]
    for ind3,u in list_user.iterrows():
        
        c1=[]
        c2=[]
        c_item=int(u['movieId'])
        list_curitem=train[train['movieId']==c_item]
        xt=pd.merge(list_item,list_curitem,how='inner',on=['userId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity2.append(sim)
        corres_rate2.append(u['rating'])
        similar_items.append(u['movieId'])
    
    corres_rate2 = [x for _,x in sorted(zip(similarity2,corres_rate2),reverse=True)]
    similar_items=[x for _,x in sorted(zip(similarity2,similar_items),reverse=True)]
    
    similarity2.sort(reverse=True)
    
    corres_rate2=corres_rate2[0:K]
    similarity2=similarity2[0:K]
    similar_items=similar_items[0:K]
    
    
    
    ############### SUIR #########################
    suir=[]
    suir_user=[]
    suir_item=[]
    suir_ratings=[]
    for p1,p2 in zip(similarity1,similar_users):
        x1=train[train['userId']==p2]
        for q1,q2 in zip(similarity2,similar_items):
            x2=x1[x1['movieId']==q2]
            
            if(x2.shape[0]!=0):
                try:
                    sui = 1/(math.sqrt( (1/p1) ** 2 + (1/q1) ** 2 ))
                except ZeroDivisionError:
                    sui = 0
            
                suir.append(sui)
                suir_user.append(p2)
                suir_item.append(q2)
                r=x2['rating'].tolist()[0]
                suir_ratings.append(r)
    
    
    ##################rating computation###############################3
    
    
    sum1=sum(similarity1)
    if sum1!=0:
        similarity_new1=[p/sum1 for p in similarity1]
    else:
        similarity_new1=[0 for p in similarity1]
    sum2=sum(similarity2)
    if sum2!=0:
        similarity_new2=[p/sum2 for p in similarity2]
    else:
        similarity_new2=[0 for p in similarity2]
    sum3=sum(suir)
    if sum3!=0:
        similarity_new3=[p/sum3 for p in suir]
    else:
         similarity_new3=[0 for p in suir]
    pred=[[0 for u in range(10)] for t in range(10)] 
    
    for p1,p2,p3 in zip(similarity_new1,similar_users,corres_rate1):
        

        p = p3 - (user_mean[int(p2)] - user_mean[usr])
        
        for e in range(0,10):
            for d in range(0,10):
                pred[e][d]+=(p*p1*Lambda[e]*(1-delta[d]))
   
    for p1,p2,p3 in zip(similarity_new2,similar_items,corres_rate2):
        
        p = p3  - (movies_mean[int(p2)][2] - movies_mean[mov][2])
        for e in range(0,10):
            for d in range(0,10):
                pred[e][d]+=(p*p1*Lambda[e]*(1-delta[d]))
   
             
    for p1,p2,p3,p4 in zip(similarity_new3,suir_user,suir_item,suir_ratings):
        
        p = p4 - (user_mean[int(p2)] - user_mean[usr]) - (movies_mean[int(p3)][2] - movies_mean[mov][2])
        for e in range(0,10):
            for d in range(0,10):
                pred[e][d]+=(p*p1*Lambda[e]*(1-delta[d]))
   
    for e in range(0,10):
        for d in range(0,10):
            rmse[e][d]+=((actual-pred[e][d])**2)
t2=time.time()


In [11]:
index

9721

In [12]:
%matplotlib inline

In [13]:
for i in range(10):
    for j in range(10):
        rmse[i][j]=math.sqrt(rmse[i][j]/index)

In [ ]:
rmse

In [22]:
min_val=9999
min_lambda=0
min_delta=0

In [27]:
for i in range(10):
    for j in range(10):
        if min_val> rmse[i][j]:
            min_val=rmse[i][j]
            min_lambda=(i/10)
            min_delta=(j/10)

In [28]:
min_val

1.1213708942729583

In [29]:
min_lambda

0.4

In [30]:
min_delta

0.2

In [31]:
holdout.describe()

,movieId,rating,userId
count,9722.000000,9722.000000,9722.000000
mean,2720.068916,3.538675,347.228348
std,2390.148336,1.062009,195.238989
min,0.000000,0.500000,1.000000
25%,832.000000,3.000000,182.000000
50%,1954.500000,4.000000,367.000000
75%,4139.000000,4.000000,520.000000
max,9116.000000,5.000000,671.000000


In [ ]:
#test set

In [32]:

t1 = time.time()

predicted_rating=[];
K=50
Lambda =min_lambda 
delta =min_delta
rmse =  0
mae= 0
gim = 0
gpim=0
c_gim=0
c_gpim=0
for index,i in test.iterrows():
    
    usr=int(i['userId'])
    mov=int(i['movieId'])
    actual=i['rating']
    
    list_user = train[train['userId']==usr]
    list_item = train[train['movieId']==mov]
    
    ############### SUR #########################
    similarity1=[]
    corres_rate1=[]
    similar_users=[]
    for ind2,u in list_item.iterrows():
        
        c1=[]
        c2=[]
        c_user=int(u['userId'])
        list_curuser=train[train['userId']==c_user]
        xt=pd.merge(list_user,list_curuser,how='inner',on=['movieId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity1.append(sim)
        corres_rate1.append(u['rating'])
        similar_users.append(u['userId'])
    
    corres_rate1 = [x for _,x in sorted(zip(similarity1,corres_rate1),reverse=True)]
    similar_users=[x for _,x in sorted(zip(similarity1,similar_users),reverse=True)]
    
    similarity1.sort(reverse=True)
    
    corres_rate1=corres_rate1[0:K]
    similarity1=similarity1[0:K]
    similar_users=similar_users[0:K]
    
    
    ############### SIR #########################
    similarity2=[]
    corres_rate2=[]
    similar_items=[]
    for ind3,u in list_user.iterrows():
        
        c1=[]
        c2=[]
        c_item=int(u['movieId'])
        list_curitem=train[train['movieId']==c_item]
        xt=pd.merge(list_item,list_curitem,how='inner',on=['userId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity2.append(sim)
        corres_rate2.append(u['rating'])
        similar_items.append(u['movieId'])
    
    corres_rate2 = [x for _,x in sorted(zip(similarity2,corres_rate2),reverse=True)]
    similar_items=[x for _,x in sorted(zip(similarity2,similar_items),reverse=True)]
    
    similarity2.sort(reverse=True)
    
    corres_rate2=corres_rate2[0:K]
    similarity2=similarity2[0:K]
    similar_items=similar_items[0:K]
    
    
    
    ############### SUIR #########################
    suir=[]
    suir_user=[]
    suir_item=[]
    suir_ratings=[]
    for p1,p2 in zip(similarity1,similar_users):
        x1=train[train['userId']==p2]
        for q1,q2 in zip(similarity2,similar_items):
            x2=x1[x1['movieId']==q2]
            
            if(x2.shape[0]!=0):
                try:
                    sui = 1/(math.sqrt( (1/p1) ** 2 + (1/q1) ** 2 ))
                except ZeroDivisionError:
                    sui = 0
            
                suir.append(sui)
                suir_user.append(p2)
                suir_item.append(q2)
                r=x2['rating'].tolist()[0]
                suir_ratings.append(r)
    
    
    ##################rating computation###############################3
    
    
    sum1=sum(similarity1)
    if sum1!=0:
        similarity_new1=[p/sum1 for p in similarity1]
    else:
        similarity_new1=[0 for p in similarity1]
    sum2=sum(similarity2)
    if sum2!=0:
        similarity_new2=[p/sum2 for p in similarity2]
    else:
        similarity_new2=[0 for p in similarity2]
    sum3=sum(suir)
    if sum3!=0:
        similarity_new3=[p/sum3 for p in suir]
    else:
         similarity_new3=[0 for p in suir]
    pred=0 
    
    for p1,p2,p3 in zip(similarity_new1,similar_users,corres_rate1):
        

        p = p3 - (user_mean[int(p2)] - user_mean[usr])
        pred+=(p*p1*Lambda*(1-delta))
   
    for p1,p2,p3 in zip(similarity_new2,similar_items,corres_rate2):
        
        p = p3  - (movies_mean[int(p2)][2] - movies_mean[mov][2])
        pred+=(p*p1*Lambda*(1-delta))
             
    for p1,p2,p3,p4 in zip(similarity_new3,suir_user,suir_item,suir_ratings):
        
        p = p4 - (user_mean[int(p2)] - user_mean[usr]) - (movies_mean[int(p3)][2] - movies_mean[mov][2])
        pred+=(p*p1*Lambda*(1-delta))
    
    rmse+=((actual-pred)**2)
    mae+=(math.fabs(actual-pred))
    if actual >3:
        c_gim+=1
        gim+=(math.fabs(actual-pred))
    if pred > 3:
        c_gpim+=1
        gpim+=(math.fabs(actual-pred))
t2=time.time()


In [33]:
index

10030

In [34]:
rmse

12229.771950681099

In [35]:
math.sqrt(rmse/index)

1.1042278919480533

In [36]:
mae/index

0.79300729413801685

In [37]:
gim/c_gim

0.717373419029757

In [38]:
gpim/c_gpim

0.6705804504519659